In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.callbacks import TensorBoard

import math
import tensorflow as tf
import preprocessing_custom

SIZE = 1000

In [2]:
# # sample 500 images from digital, non-digital artwork, and real_world


# def sample_images():
#     # sample 500 images from digital, non-digital artwork, and real_world
#     non_digital_artwork = os.listdir('./datasets/Non-digital Artwork')
#     real_world = os.listdir('./datasets/real-world')
    
#     # digital but only diffusion
#     digitals = random.sample(os.listdir('./datasets/digital/StableDiffusion/'), math.ceil(SIZE/3))

#     # non-digitals
#     non_digitals = []
#     for folder in non_digital_artwork:
#         list = os.listdir('./datasets/Non-digital Artwork/' + folder)
#         for elem in range(len(list)):
#             list[elem] = './datasets/Non-digital Artwork/' + folder + '/' + str(list[elem])
#         random.shuffle(list)
#         non_digitals = non_digitals + list[:math.ceil(math.ceil(SIZE/3)/len(non_digital_artwork))]

#     # real_world
#     real_worlds = []
#     for folder in real_world:
#         list = os.listdir('./datasets/real-world/' + folder)
#         for elem in range(len(list)):
#             list[elem] = './datasets/real-world/' + folder + '/' + str(list[elem])
#         random.shuffle(list)
#         real_worlds = real_worlds + list[:math.ceil(math.ceil(SIZE/3)/len(real_world))]

#     # put images in dataframe
#     images = []
#     labels = []
#     for image in digitals:
#         images.append('./datasets/digital/StableDiffusion/' + image)
#         labels.append('Digital')
#     print(len(images))
#     for image in non_digitals:
#         images.append(image)
#         labels.append('Non-digital Artwork')
#     print(len(images))
#     for image in real_worlds:
#         images.append(image)
#         labels.append('Real World')
#     print(len(images))
    
#     data = {'image': images, 'label': labels}
#     df = pd.DataFrame(data)
#     data = df.sample(frac=1)[:SIZE]
#     print(len(data))
#     return data

# full_dataset = sample_images()
# print(full_dataset.head())

# X, y = preprocessing_custom.sampleImages(1000, 1000, 1000)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True)

# load X and y from keras_log_reg to save time
X = np.load('X.npy')
y = np.load('y.npy')

In [3]:
# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [4]:
# create CNN model
model = Sequential()

model.add(Conv2D(128, (3, 3), input_shape=(128, 128, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# create tensorboard
NAME = "Original_CNN"
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

early_stopping = EarlyStopping(monitor='accuracy', patience=5)


In [28]:
# # rescale images
# print("Encoding y train...")
# y_train_encoded = []
# for label in y_train:
#     if label == 'Digital':
#         y_train_encoded.append([1, 0, 0])
#     elif label == 'Non-digital Artwork':
#         y_train_encoded.append([0, 1, 0])
#     else:
#         y_train_encoded.append([0, 0, 1])
# y_train_encoded = np.asarray(y_train_encoded)

# print("Encoding y test...")
# y_test_encoded = []
# for label in y_test:
#     if label == 'Digital':
#         y_test_encoded.append([1, 0, 0])
#     elif label == 'Non-digital Artwork':
#         y_test_encoded.append([0, 1, 0])
#     else:
#         y_test_encoded.append([0, 0, 1])
# y_test_encoded = np.asarray(y_test_encoded)

# print("Rescaling train images...")
# rescaled_x_train = np.empty((0, 224, 224, 3))
# for image in X_train:
#     img = tf.keras.preprocessing.image.load_img(image, target_size=(224, 224))
#     img = tf.keras.preprocessing.image.img_to_array(img)
#     img = np.expand_dims(img, axis = 0)
#     rescaled_x_train = np.append(rescaled_x_train, img, axis = 0)

# print("Rescaling test images...")
# rescaled_x_test = np.empty((0, 224, 224, 3))
# for image in X_test:
#     img = tf.keras.preprocessing.image.load_img(image, target_size=(224, 224))
#     img = tf.keras.preprocessing.image.img_to_array(img)
#     img = np.expand_dims(img, axis = 0)
#     rescaled_x_test = np.append(rescaled_x_test, img, axis = 0)

Encoding y train...
Encoding y test...
Rescaling train images...
Rescaling test images...


In [6]:

print("Fitting model...")
model.fit(np.array(X_train), 
          y_train, 
          epochs=30,
          verbose=5, 
          callbacks=[early_stopping])

# save model
# model.save('keras_cnn.model')

# evaluate model
print("Evaluating model...")
test_loss, test_acc = model.evaluate(np.array(X_test), np.array(y_test))
print('Test accuracy:', test_acc)

Fitting model...
Epoch 1/30
Epoch 2/30
Epoch 3/30
Epoch 4/30
Epoch 5/30
Epoch 6/30
Epoch 7/30
Epoch 8/30
Epoch 9/30
Epoch 10/30
Epoch 11/30
Epoch 12/30
Epoch 13/30
Epoch 14/30
Epoch 15/30
Epoch 16/30
Epoch 17/30
Epoch 18/30
Epoch 19/30
Epoch 20/30
Epoch 21/30
Epoch 22/30
Epoch 23/30
Epoch 24/30
Epoch 25/30
Epoch 26/30
Epoch 27/30
Epoch 28/30
Epoch 29/30
Epoch 30/30
Evaluating model...
24/24 [==============================] - 10s 394ms/step - loss: 6.8773 - accuracy: 0.5320
Test accuracy: 0.5320000052452087
